In [1]:
import pandas as pd
from pathlib import Path
import datautils as du

Reading from csv


Data Exploration: https://public.tableau.com/app/profile/kate.dobinson/viz/USFFIECExploration/SummaryRatios#1


In [ ]:
firm_peers_path = "Data/FirmPeers"
df = pd.DataFrame()
for file in Path(firm_peers_path).glob("*.txt"):
    try:
        print(f"Reading file {file}")
        detail = pd.read_csv(file, sep="\t", skiprows=5)
        first_row = pd.read_csv(file, nrows=1, header=None).iloc[0,0].replace("List of Banks in Peer Group -- ", "")
        detail['Peer'] = first_row
        df = pd.concat([df, detail],axis=0)        
    except FileNotFoundError:
        print(f"File {file} not found.")
    except pd.errors.ParserError:
        print(f"Error parsing file {file}. Check if the file is tab-separated and has a header row.")
df.to_csv("Data/FirmPeers.csv", index=False)
du.write_data_to_db(df, "Peers", truncate=True)


In [ ]:
#UBPR ratio extract
du.complete_extract(download_path = 'D:/repos/Python/USCreditCard/Data/Ratios', output_path = 'D:/repos/Python/USCreditCard/Data/BulkRatios/', pattern = 'FFIEC CDR Bulk All UBPR Ratios*.zip')

In [22]:
#firm extract
firm_path = Path("Data/POR")
df = pd.DataFrame()
for file in Path(firm_path).glob("*.txt"):
    try:
        print(f"Reading file {file}")
        detail = pd.read_csv(file, sep="\t")
        first_row = pd.read_csv(file, nrows=1, header=None).iloc[0,0].replace("List of Banks in Peer Group -- ", "")
        df = pd.concat([df, detail],axis=0)        
    except FileNotFoundError:
        print(f"File {file} not found.")
    except pd.errors.ParserError:
        print(f"Error parsing file {file}. Check if the file is tab-separated and has a header row.")
df.sort_values(by=[ 'Last Date/Time Submission Updated On'], ascending=False, inplace=True)
df.drop_duplicates(subset=['IDRSSD','Financial Institution Name'], inplace=True)
df.to_csv("Data/Firms.csv", index=False)

Reading file Data\POR\FFIEC CDR Call Bulk POR 03312016.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 03312017.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 03312018.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 03312019.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 03312020.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 03312021.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 03312022.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 03312023.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 06302016.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 06302017.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 06302018.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 06302019.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 06302020.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 06302021.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 06302022.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 06302023.txt
Reading file Data\POR\FFIEC CDR Call Bulk POR 09302015.t

duplicate Financial Institution Name

In [24]:
df['Financial Institution Name'].value_counts().sort_values(ascending=False).head(10)

Financial Institution Name
FIRST STATE BANK              58
FARMERS STATE BANK            37
FIRST NATIONAL BANK           26
CITIZENS STATE BANK           25
PEOPLES BANK                  20
SECURITY STATE BANK           18
CITIZENS BANK                 18
COMMUNITY STATE BANK          17
FIRST COMMUNITY BANK          16
FARMERS AND MERCHANTS BANK    15
Name: count, dtype: int64

In [6]:
bulk_ratios_path = Path('D:/repos/Python/USCreditCard/Data/BulkRatios/')
labels = du.get_measure_user_label(Path('D:/repos/Python/USCreditCard/Data/MeasureLabels/'), '*.csv')
summary_measures = du.get_measures(bulk_ratios_path, 'FFIEC CDR UBPR Ratios Summary Ratios 2023.txt', index_cols=['ID RSSD','Reporting Period'])
off_balance = du.get_measures(bulk_ratios_path, 'FFIEC CDR UBPR Ratios Off Balance Sheet Items 2023.txt', index_cols=['ID RSSD','Reporting Period'])
credit_conc = du.get_measures(bulk_ratios_path, 'FFIEC CDR UBPR Ratios Concentrations of Credit 2023.txt', index_cols=['ID RSSD','Reporting Period'])
loan_ratios_a = du.get_measures(bulk_ratios_path, 'FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-a 2023.txt', index_cols=['ID RSSD','Reporting Period'])
loan_ratios_b = du.get_measures(bulk_ratios_path, 'FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 2023.txt', index_cols=['ID RSSD','Reporting Period'])
concepts = du.get_measures(bulk_ratios_path, 'FFIEC CDR UBPR Ratios Concept Not In Presentation 2023.txt', index_cols=['ID RSSD','Reporting Period'])
bal_dollar = du.get_measures(bulk_ratios_path, 'FFIEC CDR UBPR Ratios Balance Sheet dollar 2023.txt', index_cols=['ID RSSD','Reporting Period'])
loan_mix_a = du.get_measures(bulk_ratios_path,'FFIEC CDR UBPR Ratios Allowance and Loan Mix-a 2023.txt', index_cols=['ID RSSD','Reporting Period'])
loan_mix_b = du.get_measures(bulk_ratios_path,'FFIEC CDR UBPR Ratios Allowance and Loan Mix-b 2023.txt', index_cols=['ID RSSD','Reporting Period'])
income = du.get_measures(bulk_ratios_path,'FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann 2023.txt', index_cols=['ID RSSD','Reporting Period'])
all_measures = pd.concat([concepts,summary_measures,off_balance, credit_conc,loan_ratios_a, loan_ratios_b,loan_mix_a,loan_mix_b,income]).drop_duplicates(subset=['Measure'], keep='first')
all_measures_label = all_measures.merge(labels, how='left', left_on='Measure', right_on='UBPR_Code')
all_measures_label.drop(columns=['UBPR_Code'], inplace=True)
all_measures_label.drop_duplicates(subset=['Measure'], keep='first', inplace=True)
du.write_data_to_db(all_measures_label, 'Measures', truncate = True)


In [ ]:
summary_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Summary Ratios*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(summary_files, output_path = "Data/Generated/Ratios/", overwrite = True)         
du.write_data_using_bcp('UBPR_Ratio', 'Data/Generated/Ratios/', truncate_table = True)                      

In [ ]:
credit_concentrations_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Concentrations of Credit*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(credit_concentrations_files, 'Data/Generated/Concentrations/', overwrite = True)
du.write_data_using_bcp('UBPR_Credit_Concentration', 'Data/Generated/Concentrations/', truncate_table = True)

In [2]:
ratio_pd_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(ratio_pd_files, 'Data/Generated/RatiosPD/', overwrite = True)
du.write_data_using_bcp('UBPR_RatiosPD', 'Data/Generated/RatiosPD/', truncate_table = True)

reading: FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 2002.csv
bcp Project2.dbo.UBPR_RatiosPD in "d:/repos/Python/USCreditCard/Data/Generated/RatiosPD/FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 2002.csv"  -c -t "," -r "0x0a" -S DESKTOP-192EHDA -T -F 2
reading: FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 2003.csv
bcp Project2.dbo.UBPR_RatiosPD in "d:/repos/Python/USCreditCard/Data/Generated/RatiosPD/FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 2003.csv"  -c -t "," -r "0x0a" -S DESKTOP-192EHDA -T -F 2
reading: FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 2004.csv
bcp Project2.dbo.UBPR_RatiosPD in "d:/repos/Python/USCreditCard/Data/Generated/RatiosPD/FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 2004.csv"  -c -t "," -r "0x0a" -S DESKTOP-192EHDA -T -F 2
reading: FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 2005.csv
bcp Project2.dbo.UBPR_RatiosPD in "d:/repos/Python/USCreditCard/Data/Generated/RatiosPD/FFIEC CDR UBPR Ratios PD, Nonacc and Rest Loans-b 20

In [ ]:
##files that have additional credit related measures
misc_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Concept Not In Presentation*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(misc_files, 'Data/Generated/Misc/', sub_measures = ['UBPRB538'])
off_balance_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Off Balance Sheet Items*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(off_balance_files, output_path = "Data/Generated/Misc/", sub_measures=['UBPRE263','UBPR3815'])     
balance_dollar_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Balance Sheet dollar*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(balance_dollar_files, output_path = "Data/Generated/Misc/", sub_measures=['UBPRD665','UBPR3123'])   
loan_mixa_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Allowance and Loan Mix-a*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(loan_mixa_files, 'Data/Generated/Misc/', sub_measures = ['UBPRE411'])
loan_mixb_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Allowance and Loan Mix-b*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(loan_mixb_files, 'Data/Generated/Misc/', sub_measures = ['UBPRE425'])
income_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(income_files, 'Data/Generated/Misc/', sub_measures = ['UBPRE681','UBPRE682','UBPRE683','UBPRE670'])
du.write_data_using_bcp('UBPR_Credit_Card_Additions', 'Data/Generated/Misc/', truncate_table = True)                        

In [3]:
income_files = du.get_ffiec_files(Path("Data/BulkRatios/"), pattern = "FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann*.txt", index_cols=['ID RSSD','Reporting Period'])
du.save_ffiec_ubpr_data(income_files, 'Data/Generated/Misc/', sub_measures = ['UBPRE681','UBPRE682','UBPRE683','UBPRE670'])


In [4]:
du.write_data_using_bcp('UBPR_Credit_Card_Additions', 'Data/Generated/Misc/', truncate_table = False)     

reading: FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann 2002.csv
bcp Project2.dbo.UBPR_Credit_Card_Additions in "d:/repos/Python/USCreditCard/Data/Generated/Misc/FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann 2002.csv"  -c -t "," -r "0x0a" -S DESKTOP-192EHDA -T -F 2
reading: FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann 2003.csv
bcp Project2.dbo.UBPR_Credit_Card_Additions in "d:/repos/Python/USCreditCard/Data/Generated/Misc/FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann 2003.csv"  -c -t "," -r "0x0a" -S DESKTOP-192EHDA -T -F 2
reading: FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann 2004.csv
bcp Project2.dbo.UBPR_Credit_Card_Additions in "d:/repos/Python/USCreditCard/Data/Generated/Misc/FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann 2004.csv"  -c -t "," -r "0x0a" -S DESKTOP-192EHDA -T -F 2
reading: FFIEC CDR UBPR Ratios Income Statement 1-Qtr-Ann 2005.csv
bcp Project2.dbo.UBPR_Credit_Card_Additions in "d:/repos/Python/USCreditCard/Data/Generated/Misc/FFIEC CDR UBPR Ratios I

In [2]:
#backup subset of credit card related data to csv and parquet
query = """SELECT [IDRSSD]
      ,[Financial Institution Name]
      ,[ReportingPeriod]
      ,[Measure]
      ,[Label]
      ,[Description]
      ,[Numeric_Value]
      ,[BankType]
    FROM [dbo].[UBPR_CreditCard_V]
    """
du.save_db_data_file(query, file_name = "UBPR_CreditCard_V",output_path= Path('D:/repos/Python/USCreditCard/Data/'))

In [3]:
#backup summary ratios to csv and parquet
query2 = """SELECT [IDRSSD]
      ,[Financial Institution Name]
      ,[ReportingPeriod]
      ,[Measure]
      ,[Label]
      ,[Description]
      ,[Numeric_Value]
      ,[BankType]
    FROM [dbo].[UBPR_Ratios_V]
    """
du.save_db_data_file(query2, file_name = "UBPR_Ratios_V",output_path= Path('D:/repos/Python/USCreditCard/Data/'))